In [15]:
from __future__ import print_function
import argparse
import os
import random
import math
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from models import Generator
from models import Discriminator
from PIL import Image

In [11]:
data_path = os.path.join('C:/Users/david/Documents/Python/data','human_face')
batchsize = 32
workers = 8
nz = 100

# folder dataset
human_dataset = dset.ImageFolder(root=data_path,
                        transform=transforms.Compose([
                            transforms.Resize(64),
                            transforms.CenterCrop(64),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                        ]))

assert human_dataset
human_dataloader = torch.utils.data.DataLoader(human_dataset, batch_size=batchsize,
                                        shuffle=True, num_workers=int(workers))

# Grab sample. Depending on dataset, may take a while
human_iterator = iter(human_dataloader)
sample = next(human_iterator)[0]

In [3]:
def image_to_vector(img, nz):
    preprocess = transforms.Compose([
        transforms.Grayscale(),
        transforms.CenterCrop(int(math.sqrt(nz)))
    ])
    vector = preprocess(img)
    # vector = torch.unsqueeze(vector,1)
    vector = torch.flatten(vector,start_dim=1,end_dim=3)
    return vector[:,:,None,None]

In [4]:
vector = image_to_vector(sample,nz)

In [27]:
netG = Generator(1,nz,3,64)
netG.load_state_dict(torch.load('C:/Users/david/Documents/GitHub/Anime-Portrait-Converter/DCGAN/output/generator_chkpts/netG_epoch_44.pth'))

<All keys matched successfully>

In [28]:
fixed_noise = torch.randn(batchsize, nz, 1, 1)
fake = netG(fixed_noise)
vutils.save_image(fake.detach(),
        'noise_output.png',
        normalize=True)

fake = netG(vector)
vutils.save_image(sample,
        'vector_input.png',
        normalize=True)
vutils.save_image(fake.detach(),
        'vector_output.png',
        normalize=True)

In [29]:
input = Image.open('vector_input.png')
input.show()
output = Image.open('vector_output.png')
output.show()